In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# This is how you would have gotten the files' content as strings


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd

# Attempt 1: Try a common encoding like 'latin-1' or 'cp1252'
file_robot = pd.read_csv('human_text.txt', sep='\t', encoding='latin-1')
file_human = pd.read_csv('robot_text.txt', sep='\t', encoding='latin-1')

# Attempt 2: If the first attempt fails, try a different common encoding
# file_robot = pd.read_csv('human_text.txt', sep='\t', encoding='cp1252')
# file_human = pd.read_csv('robot_text.txt', sep='\t', encoding='cp1252')

In [3]:
# Use the same encoding that works for pandas
with open('human_text.txt', 'r', encoding='latin-1') as f:
    file_robot_content = f.read()

with open('robot_text.txt', 'r', encoding='latin-1') as f:
    file_human_content = f.read()

In [4]:
def create_db(txt):
    lines = txt.split('\n')
    data = [{'text': line} for line in lines]
    return pd.DataFrame(data)

df_robot = create_db(file_robot_content)
df_human = create_db(file_human_content)

In [5]:
df_robot['human_v_robot'] = 0
df_human['human_v_robot'] = 1

In [6]:
df = pd.concat([df_robot, df_human])

In [7]:
df


,text,human_v_robot
0,,0
1,"Oh, thanks! I'm fine. this is an evening in my...",0
2,how do you feel today? tell me something about...,0
3,how many virtual friends have you got?,0
4,is that forbidden for you to tell the exact nu...,0
...,...,...
2359,nice ! i'm fine too,1
2360,what means m ?,1
2361,hi there ! ! how are you ? ð,1
2362,hi !,1


In [8]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

print(stop_words)

{'yourselves', "mustn't", 'about', 'just', "won't", "mightn't", 'yourself', "she'll", "couldn't", 'such', 'up', "he'd", "should've", 've', 'each', 'those', 'too', "you'll", 'below', 'all', "haven't", "i'd", 'what', 'before', 'hasn', 'its', 'off', 'our', "they'd", 'why', "she's", 'while', 'same', "they're", 'against', 're', 'into', 'i', 'myself', "aren't", 'shouldn', 'few', 'the', 'weren', 'me', 'it', "shan't", 'we', 'couldn', 'being', 'down', 'with', 'so', 'wouldn', 'this', "we've", 'during', 'has', 'didn', 'you', 'll', 'his', 'did', "wasn't", 'd', 'where', 'my', 'themselves', 'she', "i've", 'at', 's', 'through', "he's", "didn't", "he'll", "needn't", 'a', 'again', "hadn't", 'ourselves', 'been', 'that', 'above', 'after', "it's", 'theirs', 'ours', 'if', 'in', 'o', 'does', 'until', "you've", 'can', 'than', 'are', 'they', 'as', "she'd", 'when', 'm', 'shan', 'who', "you're", 'to', 'having', 'out', "that'll", 'any', 'further', 'ma', 'between', 'should', 'he', 'of', 'mustn', 'ain', "they'll",

In [11]:
replacements = {
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "I'm": "I am",
    "you're": "you are",
    "we're": "we are",
    "they're": "they are",
    "he'll": "he will",
    "she'll": "she will",
    "it'll": "it will",
    "i'll": "i will",
    "you'll": "you will",
    "we'll": "we will",
    "they'll": "they will",
    "he'd": "he would",
    "she'd": "she would",
    "it'd": "it would",
    "i'd": "i would",
    "you'd": "you would",
    "we'd": "we would",
    "they'd": "they would",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "wouldn't": "would not",
    "shouldn't": "should not",
    "mightn't": "might not",
    "mustn't": "must not",
    "aren't": "are not",
    "isn't": "is not",
    "wasn't": "was not",
    "weren't": "were not",
    "im": "i am",
    "u": "you",
    
}

In [12]:
from sklearn.model_selection import train_test_split

# ======== Define features and target ========
X = df['text']           # Features: text column
y = df['human_v_robot']  # Target: 0 = robot, 1 = human

# ======== Train-test split ========
# Use stratify=y to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% test data
    random_state=42,    # for reproducibility
    stratify=y          # maintain proportion of classes
)

# Print shapes to confirm
print("Training set size:", X_train.shape, y_train.shape)
print("Test set size:", X_test.shape, y_test.shape)


Training set size: (5672,) (5672,)
Test set size: (1419,) (1419,)


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Text preprocessing function
import re

def preprocess_text(text):
    text = text.lower()
    for key, val in replacements.items():
        text = text.replace(key, val)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # remove punctuation
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(preprocess_text)

# 2. Convert text to features using TF-IDF (character n-grams work well for typing patterns
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2,4))
X = vectorizer.fit_transform(df['text'])
y = df['human_v_robot']

# 3. Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

# 5. Evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy*100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Model Accuracy: 78.58%

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.97      0.85       921
           1       0.88      0.45      0.60       498

    accuracy                           0.79      1419
   macro avg       0.82      0.71      0.73      1419
weighted avg       0.81      0.79      0.76      1419



In [17]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   - -------------------------------------- 2.1/56.8 MB 9.8 MB/s eta 0:00:06
   --- ------------------------------------ 4.5/56.8 MB 10.3 MB/s eta 0:00:06
   ---- ----------------------------------- 6.8/56.8 MB 10.8 MB/s eta 0:00:05
   ------ --------------------------------- 9.2/56.8 MB 10.8 MB/s eta 0:00:05
   -------- ------------------------------- 11.5/56.8 MB 10.9 MB/s eta 0:00:05
   --------- ------------------------------ 13.9/56.8 MB 11.0 MB/s eta 0:00:04
   ----------- ---------------------------- 16.3/56.8 MB 11.1 MB/s eta 0:00:04
   ------------- -------------------------- 18.9/56.8 MB 11.2 MB/s eta 0:00:04
   -------------- ------------------------- 21.2/56.8 MB 11.2 MB/s eta 0:00:04
   ---------------- ----------------------- 23.6/56.8 MB 11.3 MB/s eta 0:00:03
   ------------------ --------------------- 26.2/56.8 MB 11.4 MB/s 


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
# ======== Imports ========
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import xgboost as xgb

# ======== Load your dataset ========
# Replace this with your actual dataset
# Assume X = features, y = target
# Example: X = df.drop('target', axis=1), y = df['target']

# X, y = ... (your dataset here)

# ======== Train-test split ========
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ======== Handle class imbalance with SMOTE ========
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE: {np.bincount(y_train)}")
print(f"After SMOTE: {np.bincount(y_res)}")

# ======== Train XGBoost classifier ========
# Calculate scale_pos_weight for imbalance
scale = len(y_res[y_res==0]) / len(y_res[y_res==1])

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

xgb_model.fit(X_res, y_res)

# ======== Make predictions ========
# Get predicted probabilities
y_probs = xgb_model.predict_proba(X_test)[:,1]

# Adjust threshold to improve recall for minority class
threshold = 0.3  # Lower than default 0.5
y_pred = (y_probs >= threshold).astype(int)

# ======== Evaluation ========
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_probs))


Before SMOTE: [3781 1891]
After SMOTE: [3781 3781]


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [11:50:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Confusion Matrix:
 [[545 401]
 [ 57 416]]

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.58      0.70       946
           1       0.51      0.88      0.64       473

    accuracy                           0.68      1419
   macro avg       0.71      0.73      0.67      1419
weighted avg       0.77      0.68      0.68      1419

ROC AUC Score: 0.8433014942184518


In [23]:
# ======== Imports ========
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import xgboost as xgb
import re

# ======== Load your dataset ========
# Assuming df already exists with columns: 'text' and 'human_v_robot'
# df = pd.read_csv("your_dataset.csv")

# ======== Text preprocessing ========
# Contraction replacements


def clean_text(text):
    text = text.lower()
    for k, v in replacements.items():
        text = text.replace(k, v)
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()

df['text'] = df['text'].astype(str).apply(clean_text)

# ======== Define features and target ========
X = df['text']
y = df['human_v_robot']

# ======== Train-test split ========
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ======== TF-IDF Vectorization ========
vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=5000,
    ngram_range=(1,2)  # unigrams + bigrams
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# ======== Handle class imbalance with partial SMOTE ========
smote = SMOTE(sampling_strategy=0.7, random_state=42)  # partial oversample
X_res, y_res = smote.fit_resample(X_train_vec, y_train)

print(f"Before SMOTE: {np.bincount(y_train)}")
print(f"After SMOTE: {np.bincount(y_res)}")

# ======== Train XGBoost and RandomForest ========
scale = len(y_res[y_res==0]) / len(y_res[y_res==1])

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale,
    eval_metric='logloss',
    random_state=42
)

rf_model = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    random_state=42
)

# ======== Voting Ensemble ========
ensemble = VotingClassifier(
    estimators=[('xgb', xgb_model), ('rf', rf_model)],
    voting='soft'  # use predicted probabilities
)

ensemble.fit(X_res, y_res)

# ======== Predict probabilities ========
y_probs = ensemble.predict_proba(X_test_vec)[:,1]

# ======== Automatic Threshold Optimization ========
precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-6)
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
print("Optimal threshold:", best_threshold)

# Predict using optimized threshold
y_pred_opt = (y_probs >= best_threshold).astype(int)

# ======== Evaluation ========
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_opt))
print("\nClassification Report:\n", classification_report(y_test, y_pred_opt))
print("Accuracy:", accuracy_score(y_test, y_pred_opt))
print("ROC AUC Score:", roc_auc_score(y_test, y_probs))


Before SMOTE: [3781 1891]
After SMOTE: [3781 2646]
Optimal threshold: 0.4333497692252626
Confusion Matrix:
 [[780 166]
 [ 84 389]]

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.82      0.86       946
           1       0.70      0.82      0.76       473

    accuracy                           0.82      1419
   macro avg       0.80      0.82      0.81      1419
weighted avg       0.84      0.82      0.83      1419

Accuracy: 0.8238195912614518
ROC AUC Score: 0.8869838063013735


In [22]:
import joblib
joblib.dump(ensemble, "typing_model.pkl")


['typing_model.pkl']